In [1]:
import pandas as pd
import numpy as np
import pathmagic
import initialise as init
import glob
import sys,os
from functools import reduce

In [32]:
pd.set_option('display.max_columns', 100)

In [ ]:
1. Load data
2. Merge msoa together and lsoa together

## Load census data

In [163]:
# get list of census files to load

#10, 14 removed
census_files_to_load = ['01','02','03','04','05','06','07','08','09','10','11','12','13','15','16','17','18','19','20', #demography and migration
                        '21','30', #ethnicity and religion
                        '61','62','63','64','65','66', #work and travel
                        '44','45','46','47','48','50','51','52','53','54','55','56', #housing
                        '77','78', #sex orient and gender
                        '67','68' #educ
                       ]
# census_files_to_load = ['01','02','07']


census_path = init.raw_path + "/census_data/"

census_files_paths = []

area_code_dict = {'lsoa':[], 'msoa':[]}

index_file_counter = 0

for x in census_files_to_load:
    file_to_append = census_path + "census2021-ts0" + x + "/census2021-ts0" + x + "-lsoa.csv"
    append_file = True
    
    if os.path.exists(file_to_append) == False:
        file_to_append = file_to_append.replace('lsoa', 'msoa')
        if os.path.exists(file_to_append) == False:
            print(x, " has no lsoa or msoa group")
            append_file = False  
        else:
            print(x, " has no lsoa group, so msoa group used")
            area_code_dict['msoa'].append(index_file_counter)
            
    else:
        area_code_dict['lsoa'].append(index_file_counter)
    if append_file:
        census_files_paths.append(file_to_append)
        index_file_counter +=1

    
census_files_paths

area_code_dict

07  has no lsoa group, so msoa group used
09  has no lsoa or msoa group
10  has no lsoa group, so msoa group used
12  has no lsoa or msoa group
13  has no lsoa group, so msoa group used
64  has no lsoa group, so msoa group used
47  has no lsoa group, so msoa group used
48  has no lsoa group, so msoa group used
77  has no lsoa group, so msoa group used
78  has no lsoa group, so msoa group used


{'lsoa': [0,
  1,
  2,
  3,
  4,
  5,
  7,
  9,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  23,
  24,
  25,
  26,
  27,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  39,
  40],
 'msoa': [6, 8, 10, 22, 28, 29, 37, 38]}

In [376]:
# load census data
census_dfs = []
for x in census_files_paths:
    census_dfs.append(pd.read_csv(x))

### Cleaning census data

In [355]:
# get common cols
census_cols = [df.columns for df in census_dfs]
all_cols_set = set([x for sublist in census_cols for x in sublist])
all_cols = [x for sublist in census_cols for x in sublist]

all_cols_dict = dict()
    
for unique_col in all_cols_set:
    all_cols_dict[unique_col] = all_cols.count(unique_col)
    
two_cols = []
for key, value in all_cols_dict.items():
    if value ==2:
        print(key, value)
        two_cols.append(key)
        
for key, value in all_cols_dict.items():
    if value ==41:
        print(key, value)
# communal cols in all dfs are: 'geography code' 'geography' 'date' 

propensity_Passports held: Africa: North Africa 2
propensity_Passports held: Middle East and Asia: South-East Asia 2
propensity_Passports held: British Overseas Territories 2
propensity_Passports held: Europe: Other Europe: EU Member countries 2
propensity_Passports held: Europe: Other Europe: EU Member countries: Portugal 2
propensity_Passports held: Middle East and Asia: Middle East 2
propensity_Passports held: Europe: Other Europe: Rest of Europe: Turkey 2
propensity_Passports held: The Americas and the Caribbean 2
propensity_Passports held: Europe: Other Europe: EU Member countries: Other EU countries 2
propensity_Passports held: Africa: Central and Western Africa 2
propensity_Sex: Male 2
propensity_Passports held: Europe: Other Europe: EU Member countries: Lithuania 2
propensity_Passports held: Europe: Ireland 2
propensity_Passports held: Africa 2
propensity_Passports held: Europe: Other Europe 2
propensity_Passports held: Europe: Other Europe: EU Member countries: Germany 2
prope

In [354]:
def clean_census_df(df):

#     total_col always 4th and 5+ are value cols
    total_col = df.columns[3]
    value_cols = list(df.columns[4:])
    rename_cols_dict = dict()

    for value_col in value_cols:
        try:
            df[value_col] = df[value_col] / df[total_col]
        except ZeroDivisionError:
            df[value_col] = 0
        df[value_col].fillna(0,inplace = True)
        
#     rename newly created propensity cols
        if value_col[-17:] == '; measures: Value':
            rename_cols_dict[value_col] = "propensity_" + value_col[:len(value_col) - 17]
        else:
            rename_cols_dict[value_col] = "propensity_" + value_col
            
    df.drop([total_col, 'date', 'geography'], inplace = True, axis = 1)

    df['geography code'] = df['geography code'].astype(str)
    df.rename(columns = rename_cols_dict, inplace = True)
    return df


lsoa_dfs = []
msoa_dfs = []
for i, census_df in enumerate(census_dfs):
    if i in area_code_dict['lsoa']:
        lsoa_dfs.append(clean_census_df(census_df))
        
    elif i in area_code_dict['msoa']:
        msoa_dfs.append(clean_census_df(census_df))
        
    else:
        print(i, ' not done')


In [356]:
census_dfs[0]

,geography code,propensity_Residence type: Lives in a household,propensity_Residence type: Lives in a communal establishment
0,E01011954,1.000000,0.000000
1,E01011969,1.000000,0.000000
2,E01011970,1.000000,0.000000
3,E01011971,1.000000,0.000000
4,E01033465,1.000000,0.000000
...,...,...,...
35667,W01001300,0.993092,0.006908
35668,W01001301,1.000000,0.000000
35669,W01001308,1.000000,0.000000
35670,W01001309,1.000000,0.000000


In [357]:
# merge lsoa and msoa dfs together respectively

lsoa_df = reduce(lambda  left,right: pd.merge(left,right,on=['geography code'],
                                            how='outer'), lsoa_dfs)

# check if any cols have missings
[x for x in lsoa_df.columns if lsoa_df[x].isna().sum() >0]

# remove the _y columns where duplicates - _ys seem to be poorly populated
for x in list(lsoa_df.columns):
    if x[-2:] == "_y":
        print(x)
        lsoa_df.drop(x, inplace = True, axis = 1)

propensity_Sex: Female_y
propensity_Sex: Male_y


In [358]:
msoa_df = reduce(lambda  left,right: pd.merge(left,right,on=['geography code'],
                                            how='outer'), msoa_dfs)

# check if any cols have missings
print([x for x in msoa_df.columns if msoa_df[x].isna().sum() >0])

# analysis done shows that there are two columns one at lsoa level and one at msoa level - remove the one at msoa level as both equally populated
for x in list(msoa_df.columns):
    if x in two_cols:
        print(x)
        msoa_df.drop(x, inplace = True, axis = 1)

[]
propensity_Passports held: Europe
propensity_Passports held: Europe: United Kingdom
propensity_Passports held: Europe: Ireland
propensity_Passports held: Europe: Other Europe
propensity_Passports held: Europe: Other Europe: EU Member countries
propensity_Passports held: Europe: Other Europe: EU Member countries: France
propensity_Passports held: Europe: Other Europe: EU Member countries: Germany
propensity_Passports held: Europe: Other Europe: EU Member countries: Italy
propensity_Passports held: Europe: Other Europe: EU Member countries: Portugal
propensity_Passports held: Europe: Other Europe: EU Member countries: Spain
propensity_Passports held: Europe: Other Europe: EU Member countries: Lithuania
propensity_Passports held: Europe: Other Europe: EU Member countries: Poland
propensity_Passports held: Europe: Other Europe: EU Member countries: Romania
propensity_Passports held: Europe: Other Europe: EU Member countries: Other EU countries
propensity_Passports held: Europe: Other Eu

## Create starting file

In [359]:
# create starting file (all unique lsoa and msoa codes) - required to join msoa to lsoa
master_areacode_file_name = "PCD_OA_LSOA_MSOA_LAD_MAY22_UK_LU.csv"

master_areacode_file = pd.read_csv(census_path + master_areacode_file_name, usecols = ['lsoa11cd', 'msoa11cd'],  encoding = "ISO-8859-1")
master_areacode_file = master_areacode_file[~master_areacode_file.duplicated(subset = 'lsoa11cd', keep = 'first')]
master_areacode_file.dropna(axis = 0, inplace = True)

# attempted to download other losao msoa files (pre 2022) but none added any additional codes to this one

In [360]:
master_areacode_file['lsoa11cd'] = master_areacode_file['lsoa11cd'].astype(str)
master_areacode_file['msoa11cd'] = master_areacode_file['msoa11cd'].astype(str)

In [361]:
master_lsoa = set(master_areacode_file['lsoa11cd'].unique())
len(master_lsoa)

42621

In [362]:
census_lsoa = set(census_dfs[0]['geography code'].unique())
len(census_lsoa)

35672

In [363]:
missing_by_country = dict()
missing_by_country['E'] = len([x for x in census_lsoa - master_lsoa if x[0] == "E"])
missing_by_country['S'] = len([x for x in census_lsoa - master_lsoa if x[0] == "S"])
missing_by_country['W'] = len([x for x in census_lsoa - master_lsoa if x[0] == "W"])
missing_by_country['N'] = len([x for x in census_lsoa - master_lsoa if x[0] == "N"])
missing_by_country['other'] = len([x for x in census_lsoa - master_lsoa if x[0] not in ['E','S','W',"N"]])
missing_by_country

{'E': 1945, 'S': 0, 'W': 80, 'N': 0, 'other': 0}

* some lsoa's in the census data are not accounted for in the overarching file - in Wales and England only - expedcted as census data is england and wales only
* no lookups for these codes exist online (extensive research done - as far as downloading Ordinance survey free trial data and seeing if any are in there - could only download small area in free trial)

## Combine data

In [368]:
# merge together - left join (removes 2000 lsoas in census data not in address file
merged_df = master_areacode_file.merge(lsoa_df, left_on = 'lsoa11cd', right_on = 'geography code', how = 'left', )
merged_df = merged_df.merge(msoa_df, left_on = 'msoa11cd', right_on = 'geography code', how = 'left')

In [369]:
merged_df.drop(['geography code_x', 'geography code_y'], inplace = True, axis = 1)

In [370]:
merged_df.isnull().sum()

lsoa11cd                                                                                                                0
msoa11cd                                                                                                                0
propensity_Residence type: Lives in a household                                                                      8974
propensity_Residence type: Lives in a communal establishment                                                         8974
propensity_Marital and civil partnership status: Never married and never registered a civil partnership              8974
                                                                                                                     ... 
propensity_Gender identity: Gender identity different from sex registered at birth but no specific identity given    8573
propensity_Gender identity: Trans woman                                                                              8573
propensity_Gender identi

In [372]:
# Known missings: impute with -1
merged_df.fillna(-1, inplace = True)

In [373]:
merged_df.describe()

,propensity_Residence type: Lives in a household,propensity_Residence type: Lives in a communal establishment,propensity_Marital and civil partnership status: Never married and never registered a civil partnership,propensity_Marital and civil partnership status: Married or in a registered civil partnership,propensity_Marital and civil partnership status: Married or in a registered civil partnership: Married,propensity_Marital and civil partnership status: Married or in a registered civil partnership: Married: Opposite sex,propensity_Marital and civil partnership status: Married or in a registered civil partnership: Married: Same sex,propensity_Marital and civil partnership status: Married or in a registered civil partnership: In a registered civil partnership,propensity_Marital and civil partnership status: Married or in a registered civil partnership: In a registered civil partnership: Opposite sex,propensity_Marital and civil partnership status: Married or in a registered civil partnership: In a registered civil partnership: Same sex,"propensity_Marital and civil partnership status: Separated, but still legally married or still legally in a civil partnership","propensity_Marital and civil partnership status: Separated, but still legally married or still legally in a civil partnership: Separated, but still married","propensity_Marital and civil partnership status: Separated, but still legally married or still legally in a civil partnership: Separated, but still in a registered civil partnership",propensity_Marital and civil partnership status: Divorced or civil partnership dissolved,propensity_Marital and civil partnership status: Divorced or civil partnership dissolved: Divorced,propensity_Marital and civil partnership status: Divorced or civil partnership dissolved: Formerly in a civil partnership now legally dissolved,propensity_Marital and civil partnership status: Widowed or surviving civil partnership partner,propensity_Marital and civil partnership status: Widowed or surviving civil partnership partner: Widowed,propensity_Marital and civil partnership status: Widowed or surviving civil partnership partner: Surviving partner from civil partnership,propensity_Household composition: One person household,propensity_Household composition: One person household: Aged 66 years and over,propensity_Household composition: One person household: Other,propensity_Household composition: Single family household,propensity_Household composition: Single family household: All aged 66 years and over,propensity_Household composition: Single family household: Married or civil partnership couple,propensity_Household composition: Single family household: Married or civil partnership couple: No children,propensity_Household composition: Single family household: Married or civil partnership couple: Dependent children,propensity_Household composition: Single family household: Married or civil partnership couple: All children non-dependent,propensity_Household composition: Single family household: Cohabiting couple family,propensity_Household composition: Single family household: Cohabiting couple family: No children,propensity_Household composition: Single family household: Cohabiting couple family: With dependent children,propensity_Household composition: Single family household: Cohabiting couple family: All children non-dependent,propensity_Household composition: Single family household: Lone parent family,propensity_Household composition: Single family household: Lone parent family: With dependent children,propensity_Household composition: Single family household: Lone parent family: All children non-dependent,propensity_Household composition: Single family household: Other single family household,propensity_Household composition: Single family household: Other single family household: Other family composition,propensity_Household composition: Other household types,propensity_Household composition: Other household types: With dependent child

In [379]:
merged_df.head()

,lsoa11cd,msoa11cd,propensity_Residence type: Lives in a household,propensity_Residence type: Lives in a communal establishment,propensity_Marital and civil partnership status: Never married and never registered a civil partnership,propensity_Marital and civil partnership status: Married or in a registered civil partnership,propensity_Marital and civil partnership status: Married or in a registered civil partnership: Married,propensity_Marital and civil partnership status: Married or in a registered civil partnership: Married: Opposite sex,propensity_Marital and civil partnership status: Married or in a registered civil partnership: Married: Same sex,propensity_Marital and civil partnership status: Married or in a registered civil partnership: In a registered civil partnership,propensity_Marital and civil partnership status: Married or in a registered civil partnership: In a registered civil partnership: Opposite sex,propensity_Marital and civil partnership status: Married or in a registered civil partnership: In a registered civil partnership: Same sex,"propensity_Marital and civil partnership status: Separated, but still legally married or still legally in a civil partnership","propensity_Marital and civil partnership status: Separated, but still legally married or still legally in a civil partnership: Separated, but still married","propensity_Marital and civil partnership status: Separated, but still legally married or still legally in a civil partnership: Separated, but still in a registered civil partnership",propensity_Marital and civil partnership status: Divorced or civil partnership dissolved,propensity_Marital and civil partnership status: Divorced or civil partnership dissolved: Divorced,propensity_Marital and civil partnership status: Divorced or civil partnership dissolved: Formerly in a civil partnership now legally dissolved,propensity_Marital and civil partnership status: Widowed or surviving civil partnership partner,propensity_Marital and civil partnership status: Widowed or surviving civil partnership partner: Widowed,propensity_Marital and civil partnership status: Widowed or surviving civil partnership partner: Surviving partner from civil partnership,propensity_Household composition: One person household,propensity_Household composition: One person household: Aged 66 years and over,propensity_Household composition: One person household: Other,propensity_Household composition: Single family household,propensity_Household composition: Single family household: All aged 66 years and over,propensity_Household composition: Single family household: Married or civil partnership couple,propensity_Household composition: Single family household: Married or civil partnership couple: No children,propensity_Household composition: Single family household: Married or civil partnership couple: Dependent children,propensity_Household composition: Single family household: Married or civil partnership couple: All children non-dependent,propensity_Household composition: Single family household: Cohabiting couple family,propensity_Household composition: Single family household: Cohabiting couple family: No children,propensity_Household composition: Single family household: Cohabiting couple family: With dependent children,propensity_Household composition: Single family household: Cohabiting couple family: All children non-dependent,propensity_Household composition: Single family household: Lone parent family,propensity_Household composition: Single family household: Lone parent family: With dependent children,propensity_Household composition: Single family household: Lone parent family: All children non-dependent,propensity_Household composition: Single family household: Other single family household,propensity_Household composition: Single family household: Other single family household: Other family composition,propensity_Household composition: Other household types,propensity_Household composition: Other household types: Wi

In [381]:
# Save census data
merged_df.to_pickle(init.processed_path + "/census_data.pickle" ,compression = 'gzip', protocol = 4, )